In [1]:
import numpy as np
import pandas as pd
import pygrib
from collections import defaultdict

In [ ]:
import pygrib
filename = "f6bf196085042efe21fd5dd7cad85a15.grib"
grbs = pygrib.open(filename)
print(sum(1 for _ in grbs))


744


In [16]:
grbs = pygrib.open(filename)
for i, grb in enumerate(grbs):
    print(i, grb)
    if i >= 9:
        break

0 1:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 201309300000
1 2:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 1 hrs:from 201310010000
2 3:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 2 hrs:from 201310010000
3 4:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 3 hrs:from 201310010000
4 5:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 4 hrs:from 201310010000
5 6:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 5 hrs:from 201310010000
6 7:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 6 hrs:from 201310010000
7 8:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 7 hrs:from 201310010000
8 9:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 8 hrs:from 201310010000
9 10:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201310010000


In [17]:
import pygrib
from collections import Counter

filename = "f6bf196085042efe21fd5dd7cad85a15.grib"
grbs = pygrib.open(filename)

hours_utc = []

for grb in grbs:
    if grb.name != "2 metre temperature":
        continue
    dt_utc = grb.validDate   # datetime in UTC (model time)
    hours_utc.append(dt_utc.hour)

grbs.close()

print("Unique UTC hours:", sorted(set(hours_utc)))
print("Counts per UTC hour:", Counter(hours_utc))


Unique UTC hours: [0]
Counts per UTC hour: Counter({0: 744})


In [24]:
import pygrib

grbs = pygrib.open(filename)
sample = grbs[1]
lats, lons = sample.latlons()
grbs.close()

print("lat range:", lats.min(), "to", lats.max())
print("lon range:", lons.min(), "to", lons.max())


lat range: -90.0 to 90.0
lon range: 0.0 to 359.9


In [10]:
import pygrib
import numpy as np

# --- user inputs ---
filename = "f6bf196085042efe21fd5dd7cad85a15.grib"
target_lat = 47.37   # example: Zurich latitude
target_lon = 8.55    # example: Zurich longitude
# -------------------

# Open GRIB file
grbs = pygrib.open(filename)

# Get one message to obtain the grid (lat/lon of all grid points)
sample = grbs[1]  # pygrib is 1-based indexed
lats, lons = sample.latlons()

# If longitudes are 0–360 and your lon is -180–180, or vice versa, fix here:
if lons.max() > 180 and target_lon < 0:
    target_lon = target_lon % 360  # convert e.g. -10 -> 350

# Find nearest grid point to (target_lat, target_lon)
dist_sq = (lats - target_lat)**2 + (lons - target_lon)**2
i, j = np.unravel_index(dist_sq.argmin(), dist_sq.shape)

print(f"Using grid point at lat={lats[i, j]:.3f}, lon={lons[i, j]:.3f}")

# Now loop over all 2 m temperature messages and collect values at (i, j)
temps_K = []

# Rewind iterator to the start, just in case we already iterated
grbs.rewind()

for grb in grbs:
    # You can also filter: if grb.name != '2 metre temperature': continue
    if grb.name == '2 metre temperature':
        temps_K.append(grb.values[i, j])

grbs.close()

temps_K = np.array(temps_K)

if temps_K.size == 0:
    raise RuntimeError("No 2 metre temperature values found in file.")

# Compute mean in K and convert to °C
mean_T_K = temps_K.mean()
mean_T_C = mean_T_K - 273.15

print(f"Number of time steps: {temps_K.size}")
print(f"Mean temperature at location (°C): {mean_T_C:.2f}")


Using grid point at lat=47.400, lon=8.600
Number of time steps: 744
Mean temperature at location (°C): 11.56


In [18]:
import pygrib
from collections import Counter

filename = "f6bf196085042efe21fd5dd7cad85a15.grib"
grbs = pygrib.open(filename)

hours_utc = []

for grb in grbs:
    if grb.name != "2 metre temperature":
        continue
    dt_utc = grb.validDate
    hours_utc.append(dt_utc.hour)

grbs.close()

print("Unique UTC hours:", sorted(set(hours_utc)))
print("Counts per UTC hour:", Counter(hours_utc))


Unique UTC hours: [0]
Counts per UTC hour: Counter({0: 744})


In [19]:
import pygrib
import numpy as np

filename = "f6bf196085042efe21fd5dd7cad85a15.grib"

# Zurich, Switzerland (approx)
target_lat = 47.37
target_lon = 8.55

grbs = pygrib.open(filename)

# Get lat/lon grid from first message
sample = grbs[1]
lats, lons = sample.latlons()

# Handle 0–360 vs -180–180 lon convention (probably not needed here, but safe)
if lons.max() > 180 and target_lon < 0:
    target_lon = target_lon % 360

# Nearest grid point to Zurich
dist_sq = (lats - target_lat)**2 + (lons - target_lon)**2
i_lat, j_lon = np.unravel_index(dist_sq.argmin(), dist_sq.shape)

print("Nearest grid point used for Zurich:")
print(f"  lat = {lats[i_lat, j_lon]:.3f}, lon = {lons[i_lat, j_lon]:.3f}")

temps_day_K = []
temps_night_K = []

grbs.rewind()
for idx, grb in enumerate(grbs):
    if grb.name != "2 metre temperature":
        continue

    temp_K = grb.values[i_lat, j_lon]

    # Reconstruct hour of day assuming strictly hourly data, in order
    hour_of_day = idx % 24  # 0..23

    # Define day: 06–18, night: rest (in model/UTC time)
    if 6 <= hour_of_day < 18:
        temps_day_K.append(temp_K)
    else:
        temps_night_K.append(temp_K)

grbs.close()

temps_day_K = np.array(temps_day_K)
temps_night_K = np.array(temps_night_K)

print(f"Number of daytime steps (Zurich): {temps_day_K.size}")
print(f"Number of nighttime steps (Zurich): {temps_night_K.size}")

if temps_day_K.size == 0 or temps_night_K.size == 0:
    raise RuntimeError("Still no day or night data – check that the file is really hourly and ordered.")

# Convert mean to °C
mean_day_C = temps_day_K.mean() - 273.15
mean_night_C = temps_night_K.mean() - 273.15

print(f"Mean daytime (06–18) temperature in Zurich (°C): {mean_day_C:.2f}")
print(f"Mean nighttime (18–06) temperature in Zurich (°C): {mean_night_C:.2f}")



Nearest grid point used for Zurich:
  lat = 47.400, lon = 8.600
Number of daytime steps (Zurich): 372
Number of nighttime steps (Zurich): 372
Mean daytime (06–18) temperature in Zurich (°C): 13.01
Mean nighttime (18–06) temperature in Zurich (°C): 10.11


In [ ]:
import pygrib
import numpy as np

def temp_day_night(filename, lat, lon):
    """
    Compute mean daytime (06–18) and nighttime (18–06) 2m temperature (°C)
    for a given location from hourly GRIB data.

    Skips masked values and warns if everything is masked (e.g. outside domain).
    """

    grbs = pygrib.open(filename)

    # Grid & domain info
    sample = grbs[1]
    lats, lons = sample.latlons()

    if lons.max() > 180 and lon < 0:
        lon = lon % 360

    # Nearest grid point
    dist_sq = (lats - lat)**2 + (lons - lon)**2
    i_lat, j_lon = np.unravel_index(dist_sq.argmin(), dist_sq.shape)

    print(f"Nearest grid point: lat={lats[i_lat, j_lon]:.3f}, lon={lons[i_lat, j_lon]:.3f}")

    temps_day = []
    temps_night = []

    grbs.rewind()
    for idx, grb in enumerate(grbs):
        if grb.name != "2 metre temperature":
            continue

        vals = grb.values  # this is often a masked array
        temp_K = vals[i_lat, j_lon]

        # Skip masked values
        if np.ma.is_masked(temp_K):
            continue

        hour_of_day = idx % 24  # 0..23 assuming strictly hourly sequence

        if 6 <= hour_of_day < 18:
            temps_day.append(temp_K)
        else:
            temps_night.append(temp_K)

    grbs.close()

    temps_day = np.array(temps_day, dtype=float)
    temps_night = np.array(temps_night, dtype=float)

    print(f"Unmasked daytime steps: {temps_day.size}")
    print(f"Unmasked nighttime steps: {temps_night.size}")

    if temps_day.size == 0 or temps_night.size == 0:
        raise RuntimeError(
            "All values are masked for this location (or only day or night has data). "
            "This usually means the point is outside the model domain or over a masked area."
        )

    mean_day_C = temps_day.mean() - 273.15
    mean_night_C = temps_night.mean() - 273.15

    return mean_day_C, mean_night_C


In [25]:
filename = "f6bf196085042efe21fd5dd7cad85a15.grib"
mean_day, mean_night = temp_day_night(filename, 47.37, 8.55)
print("Zurich Day:", mean_day)
print("Zurich Night:", mean_night)


Zurich Day: 13.009502049928074
Zurich Night: 10.108852599769534


In [26]:
mean_day, mean_night = temp_day_night(filename, 3.45, -76.53)
print("Cali Day:", mean_day)
print("Cali Night:", mean_night)


Cali Day: 19.892860584874313
Cali Night: 20.84689758464856


In [27]:
cities = {
    "Paris": (48.8566, 2.3522),
    "Cape Town": (-33.92, 18.42),
    "Tokyo": (35.6895, 139.6917),
}

for name, (lat, lon) in cities.items():
    d, n = temp_day_night(filename, lat, lon)
    print(name, d, n)


Paris 14.958894061016792 12.600016284245328


/var/folders/gr/pjdyrzt14_bbr3sfzt7zjf6h0000gn/T/ipykernel_84164/4191632931.py:64: UserWarning: Warning: converting a masked element to nan.
  temps_day_K = np.array(temps_day_K)
/var/folders/gr/pjdyrzt14_bbr3sfzt7zjf6h0000gn/T/ipykernel_84164/4191632931.py:65: UserWarning: Warning: converting a masked element to nan.
  temps_night_K = np.array(temps_night_K)


Cape Town nan nan
Tokyo 18.441930225331305 18.298205968385105
